# 基于CNN的MNIST手写数字识别
## 1. 加载Overlay

In [21]:
import time
from pynq import Overlay
import time
import numpy as np
from pynq import Xlnk
import struct
from scipy.misc import imread
import cv2

overlay = Overlay('./mnist_cnn.bit')
print("Overlay downloaded successfully!")

Overlay downloaded successfully!


## 2. 定义IP核驱动及其他功能函数

In [22]:
conv_ip = overlay.Conv_0
pool_ip = overlay.Pool_0
xlnk = Xlnk()
    
# 卷积IP核驱动函数    
def hwConv(conv, Kx, Ky, Sx, Sy, mode, relu_en, feat_in, W, bias, feat_out):
    conv.write(0x10, feat_in.shape[2])           # 输入特征图通道数
    conv.write(0x18, feat_in.shape[0])           # 输入特征图高度
    conv.write(0x20, feat_in.shape[1])           # 输入特征图宽度
    conv.write(0x28, feat_out.shape[2])          # 输出特征图通道数
    conv.write(0x30, Kx)                         # 卷积核高度
    conv.write(0x38, Ky)                         # 卷积核宽度
    conv.write(0x40, Sx)                         # 竖直方向上的卷积步长
    conv.write(0x48, Sy)                         # 水平方向上的卷积步长
    conv.write(0x50, mode)                       # 输入特征图是否需要padding
    conv.write(0x58, relu_en)                    # 是否使用ReLU激活函数
    conv.write(0x60, feat_in.physical_address)   # 输入特征图地址
    conv.write(0x68, W.physical_address)         # 卷积核地址
    conv.write(0x70, bias.physical_address)      # 偏置地址
    conv.write(0x78, feat_out.physical_address)  # 输出特征图地址
    conv.write(0, (conv.read(0) & 0x80) | 0x01)
    tp = conv.read(0)
    while not ((tp >> 1) & 0x1):
        tp = conv.read(0)
    
# 池化IP核驱动函数
def hwPool(pool, Kx, Ky, mode, feat_in, feat_out):
    pool.write(0x10, feat_in.shape[2])           # 输入特征图通道数
    pool.write(0x18, feat_in.shape[0])           # 输入特征图高度
    pool.write(0x20, feat_in.shape[1])           # 输入特征图宽度
    pool.write(0x28, Kx)                         # 池化核高度
    pool.write(0x30, Ky)                         # 池化核宽度
    pool.write(0x38, mode)                       # 选择池化类型（0:MEAN, 1:MIN, 2:MAX）
    pool.write(0x40, feat_in.physical_address)   # 输入特征图地址
    pool.write(0x48, feat_out.physical_address)  # 输出特征图地址
    pool.write(0, (pool.read(0) & 0x80) | 0x01)
    while not ((pool.read(0) >> 1) & 0x1):
        pass
def readbinfile(filename, size):
    f = open(filename, "rb")
    z = []
    for j in range(size):
        data = f.read(4)
        data_float = struct.unpack("f", data)[0]
        z.append(data_float)
    f.close()
    z = np.array(z, dtype = np.float32)
    return z




## 3. CNN各层输入输出的基本参数

In [23]:
##################################################
# Conv1
IN_CH1     = 1
IN_WIDTH1  = 28
IN_HEIGHT1 = 28

KERNEL_W1  = 3
STRIDE1    = 1
RELU_EN1   = 1

PADDING1   = int((KERNEL_W1 - 1)/2)

OUT_CH1     = 16
OUT_WIDTH1  = int((IN_WIDTH1  + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28
OUT_HEIGHT1 = int((IN_HEIGHT1 + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28

##################################################
# Pool1
MODE11      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH11     = OUT_CH1      # 16
IN_WIDTH11  = OUT_WIDTH1   # 28
IN_HEIGHT11 = OUT_HEIGHT1  # 28

KERNEL_W11  = 2

OUT_CH11     = IN_CH11                       # 16
OUT_WIDTH11  = int(IN_WIDTH11 /KERNEL_W11)   # 14
OUT_HEIGHT11 = int(IN_HEIGHT11/KERNEL_W11)   # 14

##################################################
# Conv2
IN_CH2     = OUT_CH11          # 16
IN_WIDTH2  = OUT_WIDTH11       # 14
IN_HEIGHT2 = OUT_HEIGHT11      # 14

KERNEL_W2  = 3
STRIDE2    = 1
RELU_EN2   = 1

PADDING2 = int((KERNEL_W2 - 1)/2)

OUT_CH2     = 32
OUT_WIDTH2  = int((IN_WIDTH2  + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14
OUT_HEIGHT2 = int((IN_HEIGHT2 + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14

##################################################
# Pool2
MODE21      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH21     = OUT_CH2       # 32
IN_WIDTH21  = OUT_WIDTH2    # 14
IN_HEIGHT21 = OUT_HEIGHT2   # 14

KERNEL_W21  = 2

OUT_CH21     = IN_CH21                      # 32
OUT_WIDTH21  = int(IN_WIDTH21 /KERNEL_W21)  # 7
OUT_HEIGHT21 = int(IN_HEIGHT21/KERNEL_W21)  # 7

##################################################
# FC1
IN_CH3     = OUT_CH21      # 32
IN_WIDTH3  = OUT_WIDTH21   # 7
IN_HEIGHT3 = OUT_HEIGHT21  # 7

KERNEL_W3  = 7
STRIDE3    = 1
RELU_EN3   = 1

OUT_CH3     = 128
OUT_WIDTH3  = int((IN_WIDTH3  - KERNEL_W3)/STRIDE3 + 1) # 1
OUT_HEIGHT3 = int((IN_HEIGHT3 - KERNEL_W3)/STRIDE3 + 1) # 1

##################################################
# FC2
IN_CH4     = OUT_CH3     # 128
IN_WIDTH4  = OUT_WIDTH3  # 1
IN_HEIGHT4 = OUT_HEIGHT3 # 1

KERNEL_W4  = 1
STRIDE4    = 1
RELU_EN4   = 1

OUT_CH4     = 10
OUT_WIDTH4  = int((IN_WIDTH4  - KERNEL_W4)/STRIDE4 + 1) # 1
OUT_HEIGHT4 = int((IN_HEIGHT4 - KERNEL_W4)/STRIDE4 + 1) # 1

## 4. 读取网络参数

In [24]:
# Input image
image = xlnk.cma_array(shape = (IN_HEIGHT1, IN_WIDTH1, IN_CH1), cacheable = 0, dtype = np.float32)

# Conv1
W_conv1 = xlnk.cma_array(shape = (KERNEL_W1, KERNEL_W1, IN_CH1, OUT_CH1), cacheable = 0, dtype = np.float32)
b_conv1 = xlnk.cma_array(shape = (OUT_CH1), cacheable = 0, dtype = np.float32)
h_conv1 = xlnk.cma_array(shape = (OUT_HEIGHT1, OUT_WIDTH1, OUT_CH1), cacheable = 0, dtype = np.float32)
h_pool1 = xlnk.cma_array(shape = (OUT_HEIGHT11, OUT_WIDTH11, OUT_CH11), cacheable = 0, dtype = np.float32)

# Conv2
W_conv2 = xlnk.cma_array(shape = (KERNEL_W2, KERNEL_W2, IN_CH2, OUT_CH2), cacheable = 0, dtype = np.float32)
b_conv2 = xlnk.cma_array(shape = (OUT_CH2), cacheable = 0, dtype = np.float32)
h_conv2 = xlnk.cma_array(shape = (OUT_HEIGHT2, OUT_WIDTH2, OUT_CH2), cacheable = 0, dtype = np.float32)
h_pool2 = xlnk.cma_array(shape = (OUT_HEIGHT21, OUT_WIDTH21, OUT_CH21), cacheable = 0, dtype = np.float32)

# FC1
W_fc1 = xlnk.cma_array(shape = (KERNEL_W3, KERNEL_W3, IN_CH3, OUT_CH3), cacheable = 0, dtype = np.float32)
b_fc1 = xlnk.cma_array(shape = (OUT_CH3), cacheable = 0, dtype = np.float32)
h_fc1 = xlnk.cma_array(shape = (OUT_HEIGHT3, OUT_WIDTH3, OUT_CH3), cacheable = 0, dtype = np.float32)

# FC2
W_fc2 = xlnk.cma_array(shape = (KERNEL_W4, KERNEL_W4, IN_CH4, OUT_CH4), cacheable = 0, dtype = np.float32)
b_fc2 = xlnk.cma_array(shape = (OUT_CH4), cacheable = 0, dtype = np.float32)
h_fc2 = xlnk.cma_array(shape = (OUT_HEIGHT4, OUT_WIDTH4, OUT_CH4), cacheable = 0, dtype = np.float32)

# Read weights and bias from pre-tranined file
print("Conv1:\tloading weight... ", end = "")

w_conv1 = readbinfile("./data/W_conv1.bin", KERNEL_W1*KERNEL_W1*IN_CH1*OUT_CH1)
w_conv1 = w_conv1.reshape((KERNEL_W1, KERNEL_W1, IN_CH1, OUT_CH1))
xlnk.cma_memcopy(W_conv1, w_conv1, KERNEL_W1*KERNEL_W1*IN_CH1*OUT_CH1*4)

print("done")
print("\tloading bias... ", end = "")
                
B_conv1 = readbinfile("./data/b_conv1.bin", OUT_CH1)
xlnk.cma_memcopy(b_conv1, B_conv1, OUT_CH1*4)

print("done")
print("Conv2:\tloading weight... ", end = "")

w_conv2 = readbinfile("./data/W_conv2.bin", KERNEL_W2*KERNEL_W2*IN_CH2*OUT_CH2)
w_conv2 = w_conv2.reshape((KERNEL_W2, KERNEL_W2, IN_CH2, OUT_CH2))
xlnk.cma_memcopy(W_conv2, w_conv2, KERNEL_W2*KERNEL_W2*IN_CH2*OUT_CH2*4)

print("done")
print("\tloading bias... ", end = "")

B_conv2 = readbinfile("./data/b_conv2.bin",OUT_CH2)
xlnk.cma_memcopy(b_conv2, B_conv2, OUT_CH1*4)

print("done")
print("FC1:\tloading weight... ", end = "")

w_fc1 = readbinfile("./data/W_fc1.bin", KERNEL_W3*KERNEL_W3*IN_CH3*OUT_CH3)
w_fc1 = w_fc1.reshape((KERNEL_W3, KERNEL_W3, IN_CH3, OUT_CH3))
xlnk.cma_memcopy(W_fc1, w_fc1, KERNEL_W3*KERNEL_W3*IN_CH3*OUT_CH3*4)

print("done")
print("\tloading bias... ", end = "")

B_fc1 = readbinfile("./data/b_fc1.bin", OUT_CH3)
xlnk.cma_memcopy(b_fc1, B_fc1, OUT_CH3*4)

print("done")
print("FC2:\tloading weight... ", end = "")

w_fc2 = readbinfile("./data/W_fc2.bin", KERNEL_W4*KERNEL_W4*IN_CH4*OUT_CH4)
w_fc2 = w_fc2.reshape((KERNEL_W4, KERNEL_W4, IN_CH4, OUT_CH4))
xlnk.cma_memcopy(W_fc2, w_fc2, KERNEL_W4*KERNEL_W4*IN_CH4*OUT_CH4*4)

print("done")
print("\tloading bias... ", end = "")

B_fc2 = readbinfile("./data/b_fc2.bin",OUT_CH4)
xlnk.cma_memcopy(b_fc2, B_fc2, OUT_CH4*4)

print("done")
print("CNN loaded successfully!")

Conv1:	loading weight... done
	loading bias... done
Conv2:	loading weight... done
	loading bias... done
FC1:	loading weight... done
	loading bias... done
FC2:	loading weight... done
	loading bias... done
CNN loaded successfully!


## 5. CNN软件推导

In [ ]:
def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        # 读取文件头
        magic_number, num_images, num_rows, num_cols = struct.unpack(">IIII", f.read(16))
        # 读取图像数据
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        # 重新调整图像数据的形状
        images = image_data.reshape(num_images, num_rows, num_cols)
        return images

def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        # 读取文件头
        magic_number, num_labels = struct.unpack(">II", f.read(8))
        # 读取标签数据
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

# 文件路径
images_path = './mnist_dataset/t10k-images.idx3-ubyte'
labels_path = './mnist_dataset/t10k-labels.idx1-ubyte'

# 加载图像和标签
images = load_mnist_images(images_path)
labels = load_mnist_labels(labels_path)

# 打印图像和标签的形状以验证加载是否正确
print(f'Images shape: {images.shape}')
print(f'Labels shape: {labels.shape}')


# 识别每张图像
right_count=0
count=0
for i in range(len(images)):
    image1 = images[i].astype(np.float32)
#     image1 = np.expand_dims(image, axis=-1)  # 将单通道图像扩展为3D张量

    for r in range(IN_HEIGHT1):
        for c in range(IN_WIDTH1):
            for ch in range(IN_CH1):
                image[r][c][ch] =  image1[r][c] #(255 - image1[r][c])/255

    # 记录开始时间
    pt0 = time.clock()

    # Conv1
    hwConv(conv_ip, KERNEL_W1, KERNEL_W1, STRIDE1, STRIDE1, 1, RELU_EN1, image, W_conv1, b_conv1, h_conv1)
    hwPool(pool_ip, KERNEL_W11, KERNEL_W11, MODE11, h_conv1, h_pool1)
    # Conv2
    hwConv(conv_ip, KERNEL_W2, KERNEL_W2, STRIDE2, STRIDE2, 1, RELU_EN2, h_pool1, W_conv2, b_conv2, h_conv2)
    hwPool(pool_ip, KERNEL_W21, KERNEL_W21, MODE21, h_conv2, h_pool2)
    # FC1
    hwConv(conv_ip, KERNEL_W3, KERNEL_W3, STRIDE3, STRIDE3, 0, RELU_EN3, h_pool2, W_fc1, b_fc1, h_fc1)
    # FC2
    hwConv(conv_ip, KERNEL_W4, KERNEL_W4, STRIDE4, STRIDE4, 0, RELU_EN4, h_fc1, W_fc2, b_fc2, h_fc2)

    # 记录结束时间
    pt1 = time.clock()
    time_hw = pt1 - pt0

    # 确定结果
    MAX = h_fc2[0][0][0]
    result = 0
    for ch in range(1, OUT_CH4):
        if h_fc2[0][0][ch] > MAX:
            MAX = h_fc2[0][0][ch]
            result = ch
    
    #统计数量
    
    print(f"Image {i} was recognized as {result}, true label is {labels[i]}")
    count=count+1;
    if result==labels[i]:
        right_count=right_count+1
    print("\t Accuary={:.5%}".format(right_count/count))

print("Recognition done.")

Images shape: (10000, 28, 28)
Labels shape: (10000,)
Image 0 was recognized as 7, true label is 7
	 Accuary=100.00000%
Image 1 was recognized as 2, true label is 2
	 Accuary=100.00000%
Image 2 was recognized as 1, true label is 1
	 Accuary=100.00000%
Image 3 was recognized as 0, true label is 0
	 Accuary=100.00000%
Image 4 was recognized as 4, true label is 4
	 Accuary=100.00000%
Image 5 was recognized as 1, true label is 1
	 Accuary=100.00000%
Image 6 was recognized as 4, true label is 4
	 Accuary=100.00000%
Image 7 was recognized as 9, true label is 9
	 Accuary=100.00000%
Image 8 was recognized as 5, true label is 5
	 Accuary=100.00000%
Image 9 was recognized as 9, true label is 9
	 Accuary=100.00000%
Image 10 was recognized as 0, true label is 0
	 Accuary=100.00000%
Image 11 was recognized as 6, true label is 6
	 Accuary=100.00000%
Image 12 was recognized as 9, true label is 9
	 Accuary=100.00000%
Image 13 was recognized as 0, true label is 0
	 Accuary=100.00000%
Image 14 was recogn

Image 122 was recognized as 7, true label is 7
	 Accuary=99.18699%
Image 123 was recognized as 6, true label is 6
	 Accuary=99.19355%
Image 124 was recognized as 7, true label is 7
	 Accuary=99.20000%
Image 125 was recognized as 9, true label is 9
	 Accuary=99.20635%
Image 126 was recognized as 0, true label is 0
	 Accuary=99.21260%
Image 127 was recognized as 5, true label is 5
	 Accuary=99.21875%
Image 128 was recognized as 8, true label is 8
	 Accuary=99.22481%
Image 129 was recognized as 5, true label is 5
	 Accuary=99.23077%
Image 130 was recognized as 6, true label is 6
	 Accuary=99.23664%
Image 131 was recognized as 6, true label is 6
	 Accuary=99.24242%
Image 132 was recognized as 5, true label is 5
	 Accuary=99.24812%
Image 133 was recognized as 7, true label is 7
	 Accuary=99.25373%
Image 134 was recognized as 8, true label is 8
	 Accuary=99.25926%
Image 135 was recognized as 1, true label is 1
	 Accuary=99.26471%
Image 136 was recognized as 0, true label is 0
	 Accuary=99.27

Image 245 was recognized as 3, true label is 3
	 Accuary=99.59350%
Image 246 was recognized as 0, true label is 0
	 Accuary=99.59514%
Image 247 was recognized as 2, true label is 4
	 Accuary=99.19355%
Image 248 was recognized as 4, true label is 4
	 Accuary=99.19679%
Image 249 was recognized as 2, true label is 2
	 Accuary=99.20000%
Image 250 was recognized as 4, true label is 4
	 Accuary=99.20319%
Image 251 was recognized as 1, true label is 1
	 Accuary=99.20635%
Image 252 was recognized as 9, true label is 9
	 Accuary=99.20949%
Image 253 was recognized as 5, true label is 5
	 Accuary=99.21260%
Image 254 was recognized as 7, true label is 7
	 Accuary=99.21569%
Image 255 was recognized as 7, true label is 7
	 Accuary=99.21875%
Image 256 was recognized as 2, true label is 2
	 Accuary=99.22179%
Image 257 was recognized as 8, true label is 8
	 Accuary=99.22481%
Image 258 was recognized as 2, true label is 2
	 Accuary=99.22780%
Image 259 was recognized as 0, true label is 6
	 Accuary=98.84

Image 368 was recognized as 6, true label is 6
	 Accuary=98.10298%
Image 369 was recognized as 3, true label is 3
	 Accuary=98.10811%
Image 370 was recognized as 7, true label is 7
	 Accuary=98.11321%
Image 371 was recognized as 2, true label is 2
	 Accuary=98.11828%
Image 372 was recognized as 0, true label is 0
	 Accuary=98.12332%
Image 373 was recognized as 8, true label is 8
	 Accuary=98.12834%
Image 374 was recognized as 8, true label is 8
	 Accuary=98.13333%
Image 375 was recognized as 5, true label is 5
	 Accuary=98.13830%
Image 376 was recognized as 4, true label is 4
	 Accuary=98.14324%
Image 377 was recognized as 1, true label is 1
	 Accuary=98.14815%
Image 378 was recognized as 1, true label is 1
	 Accuary=98.15303%
Image 379 was recognized as 4, true label is 4
	 Accuary=98.15789%
Image 380 was recognized as 0, true label is 0
	 Accuary=98.16273%
Image 381 was recognized as 3, true label is 3
	 Accuary=98.16754%
Image 382 was recognized as 3, true label is 3
	 Accuary=98.17

Image 491 was recognized as 5, true label is 5
	 Accuary=98.17073%
Image 492 was recognized as 2, true label is 2
	 Accuary=98.17444%
Image 493 was recognized as 3, true label is 3
	 Accuary=98.17814%
Image 494 was recognized as 7, true label is 7
	 Accuary=98.18182%
Image 495 was recognized as 8, true label is 8
	 Accuary=98.18548%
Image 496 was recognized as 9, true label is 9
	 Accuary=98.18913%
Image 497 was recognized as 4, true label is 4
	 Accuary=98.19277%
Image 498 was recognized as 0, true label is 0
	 Accuary=98.19639%
Image 499 was recognized as 6, true label is 6
	 Accuary=98.20000%
Image 500 was recognized as 3, true label is 3
	 Accuary=98.20359%
Image 501 was recognized as 9, true label is 9
	 Accuary=98.20717%
Image 502 was recognized as 5, true label is 5
	 Accuary=98.21074%
Image 503 was recognized as 2, true label is 2
	 Accuary=98.21429%
Image 504 was recognized as 1, true label is 1
	 Accuary=98.21782%
Image 505 was recognized as 3, true label is 3
	 Accuary=98.22

...

Image 4492 was recognized as 3, true label is 3
	 Accuary=96.77276%
Image 4493 was recognized as 7, true label is 7
	 Accuary=96.77348%
Image 4494 was recognized as 3, true label is 3
	 Accuary=96.77419%
Image 4495 was recognized as 0, true label is 0
	 Accuary=96.77491%
Image 4496 was recognized as 8, true label is 8
	 Accuary=96.77563%
Image 4497 was recognized as 7, true label is 8
	 Accuary=96.75411%
Image 4498 was recognized as 7, true label is 7
	 Accuary=96.75483%
Image 4499 was recognized as 6, true label is 6
	 Accuary=96.75556%
Image 4500 was recognized as 9, true label is 9
	 Accuary=96.75628%
Image 4501 was recognized as 3, true label is 3
	 Accuary=96.75700%
Image 4502 was recognized as 9, true label is 9
	 Accuary=96.75772%
Image 4503 was recognized as 2, true label is 2
	 Accuary=96.75844%
Image 4504 was recognized as 2, true label is 2
	 Accuary=96.75916%
Image 4505 was recognized as 9, true label is 9
	 Accuary=96.75988%
Image 4506 was recognized as 2, true label is 2


Image 4613 was recognized as 3, true label is 3
	 Accuary=96.72735%
Image 4614 was recognized as 0, true label is 0
	 Accuary=96.72806%
Image 4615 was recognized as 2, true label is 2
	 Accuary=96.72877%
Image 4616 was recognized as 9, true label is 9
	 Accuary=96.72948%
Image 4617 was recognized as 0, true label is 0
	 Accuary=96.73019%
Image 4618 was recognized as 6, true label is 6
	 Accuary=96.73089%
Image 4619 was recognized as 6, true label is 6
	 Accuary=96.73160%
Image 4620 was recognized as 6, true label is 6
	 Accuary=96.73231%
Image 4621 was recognized as 3, true label is 3
	 Accuary=96.73302%
Image 4622 was recognized as 6, true label is 6
	 Accuary=96.73372%
Image 4623 was recognized as 7, true label is 7
	 Accuary=96.73443%
Image 4624 was recognized as 7, true label is 7
	 Accuary=96.73514%
Image 4625 was recognized as 2, true label is 2
	 Accuary=96.73584%
Image 4626 was recognized as 8, true label is 8
	 Accuary=96.73655%
Image 4627 was recognized as 6, true label is 6


Image 4734 was recognized as 2, true label is 2
	 Accuary=96.74762%
Image 4735 was recognized as 9, true label is 9
	 Accuary=96.74831%
Image 4736 was recognized as 7, true label is 7
	 Accuary=96.74900%
Image 4737 was recognized as 8, true label is 8
	 Accuary=96.74968%
Image 4738 was recognized as 4, true label is 4
	 Accuary=96.75037%
Image 4739 was recognized as 0, true label is 0
	 Accuary=96.75105%
Image 4740 was recognized as 3, true label is 3
	 Accuary=96.75174%
Image 4741 was recognized as 0, true label is 0
	 Accuary=96.75243%
Image 4742 was recognized as 7, true label is 7
	 Accuary=96.75311%
Image 4743 was recognized as 8, true label is 8
	 Accuary=96.75379%
Image 4744 was recognized as 8, true label is 8
	 Accuary=96.75448%
Image 4745 was recognized as 4, true label is 4
	 Accuary=96.75516%
Image 4746 was recognized as 7, true label is 7
	 Accuary=96.75585%
Image 4747 was recognized as 7, true label is 7
	 Accuary=96.75653%
Image 4748 was recognized as 8, true label is 8


Image 4855 was recognized as 7, true label is 7
	 Accuary=96.72570%
Image 4856 was recognized as 9, true label is 9
	 Accuary=96.72637%
Image 4857 was recognized as 0, true label is 0
	 Accuary=96.72705%
Image 4858 was recognized as 1, true label is 1
	 Accuary=96.72772%
Image 4859 was recognized as 1, true label is 1
	 Accuary=96.72840%
Image 4860 was recognized as 9, true label is 4
	 Accuary=96.70850%
Image 4861 was recognized as 7, true label is 7
	 Accuary=96.70917%
Image 4862 was recognized as 0, true label is 0
	 Accuary=96.70985%
Image 4863 was recognized as 8, true label is 8
	 Accuary=96.71053%
Image 4864 was recognized as 1, true label is 1
	 Accuary=96.71120%
Image 4865 was recognized as 7, true label is 7
	 Accuary=96.71188%
Image 4866 was recognized as 4, true label is 4
	 Accuary=96.71255%
Image 4867 was recognized as 5, true label is 5
	 Accuary=96.71323%
Image 4868 was recognized as 7, true label is 7
	 Accuary=96.71390%
Image 4869 was recognized as 1, true label is 1


Image 4976 was recognized as 4, true label is 4
	 Accuary=96.70484%
Image 4977 was recognized as 1, true label is 1
	 Accuary=96.70550%
Image 4978 was recognized as 3, true label is 8
	 Accuary=96.68608%


## 5. CNN硬件推导

In [26]:
image1 = cv2.imread("./data/1.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)
for r in range(IN_HEIGHT1):
    for c in range(IN_WIDTH1):
        for ch in range(IN_CH1):
            image[r][c][ch] = (255 - image1[r][c])/255

print("Finish reading image.")

pt0 = time.clock()

# Conv1
hwConv(conv_ip, KERNEL_W1, KERNEL_W1, STRIDE1, STRIDE1, 1, RELU_EN1, image, W_conv1, b_conv1, h_conv1)
hwPool(pool_ip, KERNEL_W11, KERNEL_W11, MODE11, h_conv1, h_pool1)
# Conv2
hwConv(conv_ip, KERNEL_W2, KERNEL_W2, STRIDE2, STRIDE2, 1, RELU_EN2, h_pool1, W_conv2, b_conv2, h_conv2)
hwPool(pool_ip, KERNEL_W21, KERNEL_W21, MODE21, h_conv2, h_pool2)
# FC1
hwConv(conv_ip, KERNEL_W3, KERNEL_W3, STRIDE3, STRIDE3, 0, RELU_EN3, h_pool2, W_fc1, b_fc1, h_fc1)
# FC2
hwConv(conv_ip, KERNEL_W4, KERNEL_W4, STRIDE4, STRIDE4, 0, RELU_EN4, h_fc1, W_fc2, b_fc2, h_fc2)

pt1 = time.clock()
time_hw = pt1 - pt0

print("Hardware-accelerated inference done.")
print("Time consumed: %fs" % time_hw)

MAX = h_fc2[0][0][0]
result = 0
for ch in range(1, OUT_CH4):
    if (h_fc2[0][0][ch] > MAX):
        MAX = h_fc2[0][0][ch]
        result = ch

print("The image was recognized as " + str(result))

Finish reading image.
Hardware-accelerated inference done.
Time consumed: 1.706139s
The image was recognized as 2
